In [ ]:
# get extacly the text from image for Train
def getIdonly(CellWithoutBorder):
    graycell = cv2.cvtColor(CellWithoutBorder, cv2.COLOR_BGR2GRAY)
    threadhold = cv2.threshold(graycell,0,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    contours = find_contours(threadhold, 0.2)
    IDXmin , IDXmax , IDYmin , IDYmax = 5000,0,5000,0
    cellHeight, cellWidth = CellWithoutBorder.shape[0], CellWithoutBorder.shape[1]
    for contour in contours:
        Xmin = int(np.min(contour[:,1]))
        Xmax = int(np.max(contour[:,1]))
        Ymin = int(np.min(contour[:,0]))
        Ymax = int(np.max(contour[:,0]))
        if(abs(Ymax - Ymin) > .2*cellHeight) and (abs(Xmax - Xmin) > .05*cellWidth):
            IDXmin = min(IDXmin, Xmin)
            IDXmax = max(IDXmax, Xmax)
            IDYmin = min(IDYmin, Ymin)
            IDYmax = max(IDYmax, Ymax)

    return CellWithoutBorder[IDYmin:IDYmax,IDXmin:IDXmax]



In [ ]:
# exctract the id from the image dynamiclly by contours
def getchars2(cell):
    graycell = cv2.cvtColor(cell, cv2.COLOR_BGR2GRAY)
    threadhold = cv2.threshold(graycell,0,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    show_images([cell, threadhold],["cell","threadhold"])
    boundingboxes= []
    contours = find_contours(threadhold, 0.2)
    with_boxes  = np.zeros(cell.shape, dtype=float)
    bounding_boxes = []
    cellHeight, cellWidth = cell.shape[0], cell.shape[1]
    AVGheight, AVGwidth = 0,0
    for contour in contours:
        Xmin = int(np.min(contour[:,1]))
        Xmax = int(np.max(contour[:,1]))
        Ymin = int(np.min(contour[:,0]))
        Ymax = int(np.max(contour[:,0]))
        if( (abs(Ymax - Ymin) > .1*cellHeight) and  (abs(Xmax - Xmin) > .08*cellWidth) ):
            bounding_boxes.append([Xmin, Xmax, Ymin, Ymax])
            AVGheight += abs(Ymax - Ymin)
            AVGwidth += abs(Xmax - Xmin)

    bounding_boxes.sort(key=lambda x: x[0])
    AVGheight = AVGheight / len(contours)
    AVGwidth = AVGwidth / len(contours)
    bounding_boxes , medianwidthValue , AVGactualcel = removeReplication(bounding_boxes, AVGwidth*.5)

    afterfrombeguin2 = []
    size = cellWidth // 7
    for i in range(1,7):
        afterfrombeguin2.append([(i-1)*size,(i)*size,1,1 ])
    for box in bounding_boxes:
        boxWidth = abs(box[0] - box[1])
        NumberofChar = round(boxWidth*1.0 / medianwidthValue)
        if(boxWidth > 1.2 * medianwidthValue):
            # getCharsConnected(cell[box[2]:box[3],box[0]:box[1]])
            diff = (boxWidth) // NumberofChar
            start = box[0]
            inc = 0
            for i in range(1,NumberofChar):
                afterfrombeguin2.append([start, start+AVGactualcel+3, box[2], box[3]])
                start += AVGactualcel

            afterfrombeguin2.append([start, start + AVGactualcel, box[2], box[3]])
        else:
            afterfrombeguin2.append(box)
    
    charText=[] 
    for box in afterfrombeguin2:
        [Xmin, Xmax, Ymin, Ymax] = box
        charText.append(cell[:,Xmin:Xmax])

    return charText

In [ ]:

# exctract the id from the image stattically be assuming that id s 7 chars
def getchars(cell):
    cellHeight, cellWidth = cell.shape[0], cell.shape[1]
    afterfrombeguin2 = []
    size = cellWidth // 7
    for i in range(1,8):
        afterfrombeguin2.append([(i-1)*size,(i)*size,1,1 ])

    charText=[] 
    for box in afterfrombeguin2:
        [Xmin, Xmax, Ymin, Ymax] = box
        charText.append(cell[:,Xmin:Xmax])

    return charText

In [ ]:
# split the connected chars (not finished)
def getCharsConnected(connectedChars):
    graycell = cv2.cvtColor(connectedChars, cv2.COLOR_BGR2GRAY)
    threadhold = cv2.threshold(graycell,0,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    kernal = np.ones((3,1), np.uint8)
    threadhold = cv2.morphologyEx(threadhold, cv2.MORPH_CLOSE, kernal)
    contours = find_contours(threadhold, 0.8)
    bounding_boxes = []

    for contour in contours:
        Xmin = int(np.min(contour[:,1]))
        Xmax = int(np.max(contour[:,1]))
        Ymin = int(np.min(contour[:,0]))
        Ymax = int(np.max(contour[:,0]))
        bounding_boxes.append([Xmin, Xmax, Ymin, Ymax])

    print(bounding_boxes)

    show_images([threadhold ],["problem"])

# remove Cell black border
def removeCellBlackBorder(Borderdcell):
    cellHeight, cellWidth = Borderdcell.shape[0], Borderdcell.shape[1]
    cell = Borderdcell[int(cellHeight*.1):int(cellHeight*.9),int(cellWidth*.08):int(cellWidth*.9)]
    return cell

In [ ]:
#exctract the synbol from cell
def getSymbols(cell):
    graycell = cv2.cvtColor(cell, cv2.COLOR_BGR2GRAY)
    threadhold = cv2.threshold(graycell,0,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    kernal = np.ones((3,3), np.uint8)
    threadhold = cv2.morphologyEx(threadhold, cv2.MORPH_OPEN, kernal)
    contours = find_contours(threadhold, 0.2)
    bounding_boxes = []
    cellHeight, cellWidth = cell.shape[0], cell.shape[1]
    for contour in contours:
        Xmin = int(np.min(contour[:,1]))
        Xmax = int(np.max(contour[:,1]))
        Ymin = int(np.min(contour[:,0]))
        Ymax = int(np.max(contour[:,0]))
        if(abs(Ymax - Ymin) > .05*cellHeight) and (abs(Xmax - Xmin) > .05*cellWidth):
            bounding_boxes.append([Xmin, Xmax, Ymin, Ymax])
    bounding_boxes.sort(key=lambda x: x[0])
    charText=[] 
    for box in bounding_boxes:
        [Xmin, Xmax, Ymin, Ymax] = box
        charText.append(cell[Ymin:Ymax,Xmin:Xmax ])

    return charText

In [ ]:
# exctract the synbol extcally from cell
def getSymbolOnly(CellWithoutBorder):
    graycell = cv2.cvtColor(CellWithoutBorder, cv2.COLOR_BGR2GRAY)
    threadhold = cv2.threshold(graycell,0,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    contours = find_contours(threadhold, 0.2)
    cellHeight, cellWidth = CellWithoutBorder.shape[0], CellWithoutBorder.shape[1]
    IDXmin , IDXmax , IDYmin , IDYmax = cellWidth,0,cellHeight,0
    for contour in contours:
        Xmin = int(np.min(contour[:,1]))
        Xmax = int(np.max(contour[:,1]))
        Ymin = int(np.min(contour[:,0]))
        Ymax = int(np.max(contour[:,0]))
        IDXmin = min(IDXmin, Xmin)
        IDXmax = max(IDXmax, Xmax)
        IDYmin = min(IDYmin, Ymin)
        IDYmax = max(IDYmax, Ymax)

    if IDXmin == IDXmax: 
        IDXmax = IDXmin+1

    if IDYmin == IDYmax: 
        IDYmax = IDYmin+1
        
    return CellWithoutBorder[IDYmin:IDYmax,IDXmin:IDXmax]